In [1]:
import os, sys, csv, json, re
import numpy as np
import pandas as pd

In [2]:
station_name = 'KLAX'
airport_station_name = 'KLAX'

raw_folder = 'station-raw'
process_folder = 'station-process'
result_folder = 'station-result'

In [3]:
read_file = True
if read_file:
    file_names = os.listdir(raw_folder)
    file_names = ['KLAX.2020-10-21.csv']
    for file_name in file_names:
        if file_name.endswith('csv'):
            station_name = file_name.split('.')[0]
            out_1 = open(os.path.join(process_folder, station_name + '-meta.csv'), 'w')
            out_2 = open(os.path.join(process_folder, station_name + '-raw.csv'), 'w')
            with open(os.path.join(raw_folder, file_name), 'r') as f:
                for line in f:
                    if line.startswith('#') or line.startswith('Station') or line.startswith(','):
                        out_1.write(line)
                    if not line.startswith('#') and not line.startswith(','):
                        out_2.write(line)
            out_1.close()
            out_2.close()

In [4]:
file_name = os.path.join(process_folder, station_name + '-raw.csv')
df_raw = pd.read_csv(file_name, sep=',', parse_dates=True, infer_datetime_format=True, encoding='UTF-8')
df_raw.columns = map(str.lower,df_raw.columns)
df_raw.columns

/Users/xuanluo/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Index(['station_id', 'date_time', 'altimeter_set_1', 'air_temp_set_1',
       'dew_point_temperature_set_1', 'relative_humidity_set_1',
       'wind_speed_set_1', 'wind_direction_set_1', 'wind_gust_set_1',
       'weather_cond_code_set_1', 'cloud_layer_3_code_set_1',
       'precip_accum_one_hour_set_1', 'precip_accum_three_hour_set_1',
       'cloud_layer_1_code_set_1', 'cloud_layer_2_code_set_1',
       'precip_accum_six_hour_set_1', 'metar_set_1', 'ceiling_set_1',
       'dew_point_temperature_set_1d', 'cloud_layer_1_set_1d',
       'cloud_layer_3_set_1d', 'cloud_layer_2_set_1d', 'wind_chill_set_1d',
       'pressure_set_1d', 'heat_index_set_1d'],
      dtype='object')

In [5]:
# processed CSV header to raw CSV header
fields = {'dry_bulb':'air_temp_set_1', 
          'rel_humi':'relative_humidity_set_1', 
          'dew_point':'dew_point_temperature_set_1d', 
          'pressure':'pressure_set_1d',
          'wind_speed':'wind_speed_set_1', 
          'wind_dir':'wind_direction_set_1',
          'wind_chill': 'wind_chill_set_1d',
          'heat_index': 'heat_index_set_1d',
          'cloud_layer_1': 'cloud_layer_1_code_set_1',
#           'precip_accum_24_hour':'precip_accum_24_hour_set_1',
#           'solar_radiation':'solar_radiation_set_1',
#           'cloud_layer_1': 'cloud_layer_1_code_set_1',
#           'weather_condition': 'weather_condition_set_1d',
#           'snow_depth': 'snow_depth_set_1',
         }
fields_to_interp = ['dry_bulb', 'rel_humi', 'dew_point', 'pressure', 'wind_speed']

In [6]:
df_raw.head()

,station_id,date_time,altimeter_set_1,air_temp_set_1,dew_point_temperature_set_1,relative_humidity_set_1,wind_speed_set_1,wind_direction_set_1,wind_gust_set_1,weather_cond_code_set_1,...,precip_accum_six_hour_set_1,metar_set_1,ceiling_set_1,dew_point_temperature_set_1d,cloud_layer_1_set_1d,cloud_layer_3_set_1d,cloud_layer_2_set_1d,wind_chill_set_1d,pressure_set_1d,heat_index_set_1d
0,KLAX,2004-12-31T00:50:00Z,101437.61,14.0,NaN,71.85,2.57,170.0,NaN,NaN,...,NaN,NaN,NaN,8.96,NaN,NaN,NaN,NaN,100980.81,NaN
1,KLAX,2004-12-31T01:50:00Z,101437.61,14.0,NaN,71.85,2.06,220.0,NaN,NaN,...,NaN,NaN,NaN,8.96,NaN,NaN,NaN,NaN,100980.81,NaN
2,KLAX,2004-12-31T02:50:00Z,101437.61,13.0,NaN,87.66,4.12,170.0,NaN,NaN,...,NaN,NaN,NaN,10.98,NaN,NaN,NaN,NaN,100980.81,NaN
3,KLAX,2004-12-31T03:50:00Z,101403.90,13.0,NaN,87.66,2.57,140.0,NaN,NaN,...,NaN,NaN,NaN,10.98,NaN,NaN,NaN,NaN,100947.25,NaN
4,KLAX,2004-12-31T04:50:00Z,101437.61,13.0,NaN,87.66,3.09,80.0,NaN,NaN,...,NaN,NaN,NaN,10.98,NaN,NaN,NaN,NaN,100980.81,NaN


In [7]:
df_raw.describe()

,altimeter_set_1,air_temp_set_1,dew_point_temperature_set_1,relative_humidity_set_1,wind_speed_set_1,wind_direction_set_1,wind_gust_set_1,weather_cond_code_set_1,cloud_layer_3_code_set_1,precip_accum_one_hour_set_1,...,cloud_layer_2_code_set_1,precip_accum_six_hour_set_1,ceiling_set_1,dew_point_temperature_set_1d,cloud_layer_1_set_1d,cloud_layer_3_set_1d,cloud_layer_2_set_1d,wind_chill_set_1d,pressure_set_1d,heat_index_set_1d
count,601707.000000,601186.000000,58467.000000,601130.000000,600483.000000,592547.000000,8994.000000,84122.000000,51448.000000,13856.000000,...,146442.000000,1345.000000,187981.000000,601128.000000,0.0,0.0,0.0,13217.000000,601707.000000,14084.000000
mean,101435.542014,17.823205,11.503340,69.365425,3.285031,183.454561,10.560170,407.417584,996.045094,1.051295,...,667.106138,2.593295,1082.633528,11.274479,NaN,NaN,NaN,8.677179,100978.753684,28.393836
std,408.268935,4.113593,5.932861,19.185878,2.182879,104.973109,2.474837,3261.383094,758.121832,1.797447,...,696.526912,5.468336,1526.721805,6.037345,NaN,NaN,NaN,4.753660,406.430267,1.850983
min,97999.050000,0.000000,-21.100000,0.000000,0.000000,0.000000,7.200000,1.000000,2.000000,0.000000,...,13.000000,0.000000,0.000000,-110.700000,NaN,NaN,NaN,-2.990000,97557.740000,25.530000
25%,101185.240000,15.000000,9.400000,60.980000,1.540000,90.000000,8.750000,6.000000,402.000000,0.254000,...,183.000000,0.025000,304.800000,8.960000,NaN,NaN,NaN,4.970000,100729.580000,27.120000
50%,101422.290000,18.000000,12.800000,72.860000,3.090000,240.000000,9.770000,31.000000,703.000000,0.508000,...,353.000000,0.254000,518.160000,12.970000,NaN,NaN,NaN,8.270000,100965.560000,28.070000
75%,101693.200000,21.000000,15.600000,82.520000,4.630000,260.000000,11.830000,31.000000,1503.000000,1.270000,...,906.000000,2.286000,1066.800000,15.090000,NaN,NaN,NaN,11.680000,101235.250000,29.250000
max,103250.940000,40.000000,22.000000,100.000000,18.520000,360.000000,24.180000,199445.000000,3503.000000,25.146000,...,3503.000000,55.880000,10668.000000,22.050000,NaN,NaN,NaN,31.680000,102785.980000,38.940000


In [8]:
## Convert GTM to Local DateTime
shift_hour = -8
from datetime import timedelta
df_raw['date_time_gmt'] = df_raw['date_time']
df_raw['date_time_gmt'] = pd.to_datetime(df_raw['date_time'])
df_raw['date_time'] = df_raw['date_time_gmt'] + timedelta(hours=shift_hour)

In [9]:
# get year day hour
df_raw["year"]= df_raw['date_time'].dt.year 
df_raw["month"]= df_raw['date_time'].dt.month 
df_raw["day"]= df_raw['date_time'].dt.day 
df_raw["hour"]= df_raw['date_time'].dt.hour 
df_raw.head()

,station_id,date_time,altimeter_set_1,air_temp_set_1,dew_point_temperature_set_1,relative_humidity_set_1,wind_speed_set_1,wind_direction_set_1,wind_gust_set_1,weather_cond_code_set_1,...,cloud_layer_3_set_1d,cloud_layer_2_set_1d,wind_chill_set_1d,pressure_set_1d,heat_index_set_1d,date_time_gmt,year,month,day,hour
0,KLAX,2004-12-30 16:50:00+00:00,101437.61,14.0,NaN,71.85,2.57,170.0,NaN,NaN,...,NaN,NaN,NaN,100980.81,NaN,2004-12-31 00:50:00+00:00,2004,12,30,16
1,KLAX,2004-12-30 17:50:00+00:00,101437.61,14.0,NaN,71.85,2.06,220.0,NaN,NaN,...,NaN,NaN,NaN,100980.81,NaN,2004-12-31 01:50:00+00:00,2004,12,30,17
2,KLAX,2004-12-30 18:50:00+00:00,101437.61,13.0,NaN,87.66,4.12,170.0,NaN,NaN,...,NaN,NaN,NaN,100980.81,NaN,2004-12-31 02:50:00+00:00,2004,12,30,18
3,KLAX,2004-12-30 19:50:00+00:00,101403.90,13.0,NaN,87.66,2.57,140.0,NaN,NaN,...,NaN,NaN,NaN,100947.25,NaN,2004-12-31 03:50:00+00:00,2004,12,30,19
4,KLAX,2004-12-30 20:50:00+00:00,101437.61,13.0,NaN,87.66,3.09,80.0,NaN,NaN,...,NaN,NaN,NaN,100980.81,NaN,2004-12-31 04:50:00+00:00,2004,12,30,20


# Find year round

In [10]:
# TODO: automate the process for multiple years
year = 2019
df_year = df_raw.loc[df_raw['year'] == year]
df_year.head(5)

,station_id,date_time,altimeter_set_1,air_temp_set_1,dew_point_temperature_set_1,relative_humidity_set_1,wind_speed_set_1,wind_direction_set_1,wind_gust_set_1,weather_cond_code_set_1,...,cloud_layer_3_set_1d,cloud_layer_2_set_1d,wind_chill_set_1d,pressure_set_1d,heat_index_set_1d,date_time_gmt,year,month,day,hour
399775,KLAX,2019-01-01 00:00:00+00:00,101523.88,12.0,NaN,17.41,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,101066.69,NaN,2019-01-01 08:00:00+00:00,2019,1,1,0
399776,KLAX,2019-01-01 00:05:00+00:00,101523.88,11.0,NaN,20.15,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,101066.69,NaN,2019-01-01 08:05:00+00:00,2019,1,1,0
399777,KLAX,2019-01-01 00:10:00+00:00,101523.88,11.0,NaN,20.15,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,101066.69,NaN,2019-01-01 08:10:00+00:00,2019,1,1,0
399778,KLAX,2019-01-01 00:15:00+00:00,101523.88,11.0,NaN,18.60,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,101066.69,NaN,2019-01-01 08:15:00+00:00,2019,1,1,0
399779,KLAX,2019-01-01 00:20:00+00:00,101490.02,11.0,NaN,20.15,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,101032.99,NaN,2019-01-01 08:20:00+00:00,2019,1,1,0


In [11]:
average_data_per_hour = len(df_year['date_time']) / 8760
average_data_per_hour = max(int(round(average_data_per_hour, 1)), 1)
average_data_per_hour

13

In [12]:
df_proc = pd.DataFrame()
df_proc['date_time'] = df_year['date_time']
df_proc['month'] = df_year['month']
df_proc['day'] = df_year['day']
df_proc['hour'] = df_year['hour']
for field_name in fields:
    if fields[field_name] in df_year:
        df_proc[field_name] = df_year[fields[field_name]]

In [13]:
len(df_proc['date_time'])

114236

In [14]:
if 'cloud_layer_1' in df_proc:
    df_proc['cloud_layer_1'] = df_proc['cloud_layer_1'].fillna(0)
    df_proc['cloud_layer_1_none'] = np.where((df_proc['cloud_layer_1'] == 0) | (df_proc['cloud_layer_1'] == 1), 1, 0)

### Clean up

In [15]:
df_proc.describe()

,month,day,hour,dry_bulb,rel_humi,dew_point,pressure,wind_speed,wind_dir,wind_chill,heat_index,cloud_layer_1,cloud_layer_1_none
count,114236.000000,114236.000000,114236.000000,113827.000000,113824.000000,113824.000000,114210.000000,113653.000000,113074.000000,2995.000000,2403.000000,114236.000000,114236.000000
mean,6.515669,15.682412,11.494503,17.402723,67.817466,10.531568,101042.002697,3.377126,182.614925,7.423633,28.254253,132.999694,0.457710
std,3.452076,8.801990,6.925693,4.188274,18.695077,6.165845,371.808898,2.196935,104.153126,4.026745,1.413562,265.327308,0.498211
min,1.000000,1.000000,0.000000,4.000000,3.320000,-20.760000,99515.970000,0.000000,0.000000,-1.440000,25.530000,0.000000,0.000000
25%,4.000000,8.000000,5.000000,14.000000,59.660000,7.970000,100797.000000,2.060000,90.000000,4.080000,27.390000,1.000000,0.000000
50%,7.000000,16.000000,11.000000,17.000000,72.420000,11.970000,100999.270000,3.090000,240.000000,7.440000,28.040000,63.000000,0.000000
75%,10.000000,23.000000,18.000000,20.000000,82.010000,14.980000,101235.250000,4.630000,260.000000,10.170000,29.020000,154.000000,1.000000
max,12.000000,31.000000,23.000000,37.000000,100.000000,21.100000,102347.730000,16.980000,360.000000,22.270000,34.180000,3006.000000,1.000000


In [16]:
for field_name in fields_to_interp:
    df_proc[field_name] = df_proc[field_name].interpolate(method='linear', limit=average_data_per_hour)
df_proc.describe()

,month,day,hour,dry_bulb,rel_humi,dew_point,pressure,wind_speed,wind_dir,wind_chill,heat_index,cloud_layer_1,cloud_layer_1_none
count,114236.000000,114236.000000,114236.000000,114236.000000,114236.000000,114236.000000,114236.000000,114236.000000,113074.000000,2995.000000,2403.000000,114236.000000,114236.000000
mean,6.515669,15.682412,11.494503,17.401883,67.838282,10.536138,101042.019384,3.383287,182.614925,7.423633,28.254253,132.999694,0.457710
std,3.452076,8.801990,6.925693,4.184083,18.696527,6.162369,371.768448,2.201797,104.153126,4.026745,1.413562,265.327308,0.498211
min,1.000000,1.000000,0.000000,4.000000,3.320000,-20.760000,99515.970000,0.000000,0.000000,-1.440000,25.530000,0.000000,0.000000
25%,4.000000,8.000000,5.000000,14.000000,59.790000,7.970000,100797.000000,2.060000,90.000000,4.080000,27.390000,1.000000,0.000000
50%,7.000000,16.000000,11.000000,17.000000,72.420000,11.970000,100999.270000,3.090000,240.000000,7.440000,28.040000,63.000000,0.000000
75%,10.000000,23.000000,18.000000,20.000000,82.140000,14.980000,101235.250000,4.630000,260.000000,10.170000,29.020000,154.000000,1.000000
max,12.000000,31.000000,23.000000,37.000000,100.000000,21.100000,102347.730000,16.980000,360.000000,22.270000,34.180000,3006.000000,1.000000


In [17]:
if 'cloud_layer_1' in df_proc:
    df_proc['cloud_layer_1_digit'] = df_proc['cloud_layer_1'].astype(int) % 10
    df_proc['cloud_layer_1_digit_m'] = np.where(df_proc['cloud_layer_1_none'] == 1, np.nan, df_proc['cloud_layer_1_digit'].astype(int))

In [18]:
if 'weather_condition' in df_proc:
    df_proc['weather_condition'].unique()

In [19]:
if 'weather_condition' in df_proc:
    df_proc['is_rain'] = np.where((df_proc['weather_condition'].str.contains('Rain')) |
                                  (df_proc['weather_condition'].str.contains('Thunder')) |
                                  (df_proc['weather_condition'].str.contains('Drizzle')), 1, 0)

In [20]:
df_proc.head(5)

,date_time,month,day,hour,dry_bulb,rel_humi,dew_point,pressure,wind_speed,wind_dir,wind_chill,heat_index,cloud_layer_1,cloud_layer_1_none,cloud_layer_1_digit,cloud_layer_1_digit_m
399775,2019-01-01 00:00:00+00:00,1,1,0,12.0,17.41,-12.10,101066.69,0.0,0.0,NaN,NaN,0.0,1,0,NaN
399776,2019-01-01 00:05:00+00:00,1,1,0,11.0,20.15,-11.12,101066.69,0.0,0.0,NaN,NaN,0.0,1,0,NaN
399777,2019-01-01 00:10:00+00:00,1,1,0,11.0,20.15,-11.12,101066.69,0.0,0.0,NaN,NaN,0.0,1,0,NaN
399778,2019-01-01 00:15:00+00:00,1,1,0,11.0,18.60,-12.11,101066.69,0.0,0.0,NaN,NaN,0.0,1,0,NaN
399779,2019-01-01 00:20:00+00:00,1,1,0,11.0,20.15,-11.12,101032.99,0.0,0.0,NaN,NaN,0.0,1,0,NaN


In [21]:
df_proc.to_csv(os.path.join(process_folder, station_name + '-' + str(year) + '-p.csv'))

# Average to hour

In [22]:
columns = list(fields.keys())
if 'cloud_layer_1' in df_proc:
    columns.append('cloud_layer_1_digit_m')
if 'is_rain' in df_proc:
    columns.append('is_rain')
days_in_mon = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}

In [23]:
columns

['dry_bulb',
 'rel_humi',
 'dew_point',
 'pressure',
 'wind_speed',
 'wind_dir',
 'wind_chill',
 'heat_index',
 'cloud_layer_1',
 'cloud_layer_1_digit_m']

In [24]:
df_hourly = pd.DataFrame()
df_hourly['date_time'] = []
for col in columns:
    if col != 'weather_condition':
        df_hourly[col] = []
for m in range(1, 13):
    print('Month: ' + str(m)) 
    for d in range(1, days_in_mon[m] + 1):
        for h in range(24):
            month = str(m).zfill(2)
            day = str(d).zfill(2)
            hour = str(h).zfill(2)
            row_values = list()
            row_values.append(str(year) + '-' + month + '-' + day + '-' + hour + ':00')
            df_loc = df_proc.loc[(df_proc['month'] == int(month)) & (df_proc['day'] == int(day)) & (df_proc['hour'] == int(hour))]
            df_loc_mean = df_loc.mean(axis=0, numeric_only=True)
            df_loc_mode = df_loc.mode(axis=0)
            for col in columns:
                if col != 'weather_condition' and col != 'is_rain':
                    row_values.append(df_loc_mean[col])
            if 'is_rain' in df_loc_mode:
                if len(df_loc_mode['is_rain']) > 0:
                    row_values.append(df_loc_mode['is_rain'][0])
                else:
                    row_values.append(0)
            row_series = pd.Series(row_values, index = df_hourly.columns)
            df_hourly = df_hourly.append(row_series, ignore_index=True)

Month: 1
Month: 2
Month: 3
Month: 4
Month: 5
Month: 6
Month: 7
Month: 8
Month: 9
Month: 10
Month: 11
Month: 12


In [25]:
df_hourly.describe()

,dry_bulb,rel_humi,dew_point,pressure,wind_speed,wind_dir,wind_chill,heat_index,cloud_layer_1,cloud_layer_1_digit_m
count,8757.000000,8757.000000,8757.000000,8757.000000,8757.000000,8756.000000,477.000000,253.000000,8757.000000,7613.000000
mean,17.430496,67.718337,10.535091,101040.545492,3.383500,182.481594,7.987500,27.984867,135.027776,4.888874
std,4.174835,18.505135,6.153229,372.694867,2.076621,88.982959,4.339455,1.305415,168.578883,1.352022
min,5.892308,3.970000,-20.153077,99515.970000,0.000000,0.000000,-1.260000,25.785000,0.153846,2.000000
25%,14.307692,59.770000,8.130000,100809.969231,1.818462,93.846154,4.360000,27.113636,29.461538,3.846154
50%,17.213333,72.136923,11.956923,101001.518667,3.007692,232.307692,7.896923,27.912500,106.000000,6.000000
75%,20.230769,80.875385,14.824615,101250.813077,4.826154,256.923077,10.516667,28.597692,169.307692,6.000000
max,34.992308,100.000000,21.100000,102314.020000,15.117692,356.153846,22.270000,32.894615,2506.000000,6.000000


In [26]:
for col in columns:
    if col != 'weather_condition':
        df_hourly[col] = df_hourly[col].interpolate(method='linear', limit=3, limit_direction ='both')
df_hourly.describe()

,dry_bulb,rel_humi,dew_point,pressure,wind_speed,wind_dir,wind_chill,heat_index,cloud_layer_1,cloud_layer_1_digit_m
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,1217.000000,544.000000,8760.000000,8312.000000
mean,17.429625,67.718572,10.534542,101040.559334,3.384380,182.484126,8.818443,27.572141,135.047293,4.971097
std,4.174431,18.502089,6.152250,372.889570,2.076826,88.984463,4.710890,1.129578,168.561272,1.330571
min,5.892308,3.970000,-20.153077,99515.970000,0.000000,0.000000,-1.260000,25.785000,0.153846,2.000000
25%,14.307692,59.776346,8.134038,100809.969231,1.818462,93.846154,4.758000,26.717827,29.461538,4.000000
50%,17.207692,72.133462,11.956923,101001.691256,3.007692,232.307692,8.620435,27.681405,106.000000,6.000000
75%,20.230769,80.870810,14.824615,101250.813269,4.826154,256.923077,11.931538,28.065625,169.326923,6.000000
max,34.992308,100.000000,21.100000,102314.020000,15.117692,356.153846,22.270000,32.894615,2506.000000,6.000000


In [27]:
result_folder = 'station-result'
df_hourly.to_csv(os.path.join(result_folder, station_name + '-' + str(year) + '.csv'))

## Find nearest Airport Data - Optional

In [28]:
CSV_FILE = 'station-result/' + airport_station_name + '-' + str(year) + '.csv'
CSV_FILE_CLEAN =  'station-result/' + airport_station_name + '-' + str(year) + '-M.csv'
df_csv_result = pd.read_csv(CSV_FILE, sep=',', index_col=0, parse_dates=True, infer_datetime_format=True, encoding='UTF-8')

In [29]:
df_csv_result['cloud_layer_1_digit_m'] = df_csv_result['cloud_layer_1_digit_m'].fillna(method='ffill')
df_csv_result['cloud_layer_1_digit_m'] = df_csv_result['cloud_layer_1_digit_m'].fillna(method='bfill').astype(int)
if 'snow_depth' in df_csv_result:
    df_csv_result['snow_depth'] = df_csv_result['snow_depth'].fillna(0)

In [30]:
df_csv_result.describe()

,dry_bulb,rel_humi,dew_point,pressure,wind_speed,wind_dir,wind_chill,heat_index,cloud_layer_1,cloud_layer_1_digit_m
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,1217.000000,544.000000,8760.000000,8760.00000
mean,17.429625,67.718572,10.534542,101040.559334,3.384380,182.484126,8.818443,27.572141,135.047293,4.87226
std,4.174431,18.502089,6.152250,372.889570,2.076826,88.984463,4.710890,1.129578,168.561272,1.44512
min,5.892308,3.970000,-20.153077,99515.970000,0.000000,0.000000,-1.260000,25.785000,0.153846,2.00000
25%,14.307692,59.776346,8.134038,100809.969231,1.818462,93.846154,4.758000,26.717827,29.461538,4.00000
50%,17.207692,72.133462,11.956923,101001.691256,3.007692,232.307692,8.620435,27.681405,106.000000,6.00000
75%,20.230769,80.870810,14.824615,101250.813269,4.826154,256.923077,11.931538,28.065625,169.326923,6.00000
max,34.992308,100.000000,21.100000,102314.020000,15.117692,356.153846,22.270000,32.894615,2506.000000,6.00000


In [31]:
df_csv_result.to_csv(CSV_FILE_CLEAN)

## Fill missing values with the nearest airport (optional)

In [32]:
file_name = os.path.join(result_folder, airport_station_name + '-' + str(year) + '.csv')
df_airport = pd.read_csv(file_name, sep=',', index_col=0, parse_dates=True, infer_datetime_format=True, encoding='UTF-8')
df_airport.describe()

,dry_bulb,rel_humi,dew_point,pressure,wind_speed,wind_dir,wind_chill,heat_index,cloud_layer_1,cloud_layer_1_digit_m
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,1217.000000,544.000000,8760.000000,8312.000000
mean,17.429625,67.718572,10.534542,101040.559334,3.384380,182.484126,8.818443,27.572141,135.047293,4.971097
std,4.174431,18.502089,6.152250,372.889570,2.076826,88.984463,4.710890,1.129578,168.561272,1.330571
min,5.892308,3.970000,-20.153077,99515.970000,0.000000,0.000000,-1.260000,25.785000,0.153846,2.000000
25%,14.307692,59.776346,8.134038,100809.969231,1.818462,93.846154,4.758000,26.717827,29.461538,4.000000
50%,17.207692,72.133462,11.956923,101001.691256,3.007692,232.307692,8.620435,27.681405,106.000000,6.000000
75%,20.230769,80.870810,14.824615,101250.813269,4.826154,256.923077,11.931538,28.065625,169.326923,6.000000
max,34.992308,100.000000,21.100000,102314.020000,15.117692,356.153846,22.270000,32.894615,2506.000000,6.000000


In [33]:
file_name = os.path.join(result_folder, station_name + '-' + str(year) + '.csv')
df_station = pd.read_csv(file_name, sep=',', index_col=0, parse_dates=True, infer_datetime_format=True, encoding='UTF-8')
df_station.describe()

,dry_bulb,rel_humi,dew_point,pressure,wind_speed,wind_dir,wind_chill,heat_index,cloud_layer_1,cloud_layer_1_digit_m
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,1217.000000,544.000000,8760.000000,8312.000000
mean,17.429625,67.718572,10.534542,101040.559334,3.384380,182.484126,8.818443,27.572141,135.047293,4.971097
std,4.174431,18.502089,6.152250,372.889570,2.076826,88.984463,4.710890,1.129578,168.561272,1.330571
min,5.892308,3.970000,-20.153077,99515.970000,0.000000,0.000000,-1.260000,25.785000,0.153846,2.000000
25%,14.307692,59.776346,8.134038,100809.969231,1.818462,93.846154,4.758000,26.717827,29.461538,4.000000
50%,17.207692,72.133462,11.956923,101001.691256,3.007692,232.307692,8.620435,27.681405,106.000000,6.000000
75%,20.230769,80.870810,14.824615,101250.813269,4.826154,256.923077,11.931538,28.065625,169.326923,6.000000
max,34.992308,100.000000,21.100000,102314.020000,15.117692,356.153846,22.270000,32.894615,2506.000000,6.000000


In [34]:
for index, row in df_station.iterrows():
    for col in fields_to_interp:
        if pd.isna(row[col]):
            df_station.loc[index, col] = df_station.loc[index - 1, col] + (df_airport[col][index] - df_airport[col][index - 1])
df_station['wind_dir'] = df_station['wind_dir'].interpolate(method='linear', limit_direction ='both')
# use_airport = ['snow_depth', 'cloud_layer_1_digit_m', 'is_rain']
use_airport = ['cloud_layer_1_digit_m']
for field in use_airport:
    df_station[field] = df_airport[field]
df_station.describe()

,dry_bulb,rel_humi,dew_point,pressure,wind_speed,wind_dir,wind_chill,heat_index,cloud_layer_1,cloud_layer_1_digit_m
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,1217.000000,544.000000,8760.000000,8312.000000
mean,17.429625,67.718572,10.534542,101040.559334,3.384380,182.484126,8.818443,27.572141,135.047293,4.971097
std,4.174431,18.502089,6.152250,372.889570,2.076826,88.984463,4.710890,1.129578,168.561272,1.330571
min,5.892308,3.970000,-20.153077,99515.970000,0.000000,0.000000,-1.260000,25.785000,0.153846,2.000000
25%,14.307692,59.776346,8.134038,100809.969231,1.818462,93.846154,4.758000,26.717827,29.461538,4.000000
50%,17.207692,72.133462,11.956923,101001.691256,3.007692,232.307692,8.620435,27.681405,106.000000,6.000000
75%,20.230769,80.870810,14.824615,101250.813269,4.826154,256.923077,11.931538,28.065625,169.326923,6.000000
max,34.992308,100.000000,21.100000,102314.020000,15.117692,356.153846,22.270000,32.894615,2506.000000,6.000000


In [35]:
df_station.to_csv(os.path.join(result_folder, station_name + '-' + str(year) + '-M.csv'))